In [ ]:
# Import PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, hour, col
from pyspark.sql.types import TimestampType

In [ ]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("BWF-PySparkTask") \
    .getOrCreate()

In [ ]:
# Path to your CSV file
file_path = r'data.csv'

In [ ]:
# Read CSV into DataFrame
orders_data = spark.read.csv(file_path, header=True, inferSchema=True)

# Display the DataFrame with reordered columns
orders_data.show()

+-------------------+--------+--------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|         order_date|order_id|             product|   product_id|    category| purchase_address|quantity_ordered|  price_each|cost_price|turnover|  margin|
+-------------------+--------+--------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|2023-01-22 21:25:00|  141234|              iPhone|5638008983335|   Vêtements| """944 Walnut St|          Boston| MA 02215"""|         1|   700.0|   231.0|
|2023-01-28 14:15:00|  141235|Lightning Chargin...|5563319511488|Alimentation|  """185 Maple St|        Portland| OR 97035"""|         1|   14.95|   7.475|
|2023-01-17 13:33:00|  141236|    Wired Headphones|2113973395220|   Vêtements|  """538 Adams St|   San Francisco| CA 94016"""|         2|   11.99|   5.995|
|2023-01-05 20:33:00|  141237|    27in FHD Monitor|3069156759167

In [ ]:
# Add time_of_day column based on the hour
orders_data = orders_data.withColumn(
    "time_of_day",
    when((hour(col("order_date")) >= 5) & (hour(col("order_date")) < 12), "morning")
    .when((hour(col("order_date")) >= 12) & (hour(col("order_date")) < 18), "afternoon")
    .when((hour(col("order_date")) >= 18) & (hour(col("order_date")) < 24), "evening")
    .otherwise("night")
)

# Reorder columns to place 'time_of_day' at index 1
columns = orders_data.columns
desired_index = 1
# Remove 'time_of_day' from columns list to avoid duplication
columns.remove('time_of_day')
# Insert 'time_of_day' at the desired index
new_columns = columns[:desired_index] + ['time_of_day'] + columns[desired_index:]

# Reorder the DataFrame columns
orders_data = orders_data.select(new_columns)

# Display the DataFrame with reordered columns
orders_data.show(truncate=False)

+-------------------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|order_date         |time_of_day|order_id|product                   |product_id   |category    |purchase_address |quantity_ordered|price_each  |cost_price|turnover|margin  |
+-------------------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|2023-01-22 21:25:00|evening    |141234  |iPhone                    |5638008983335|Vêtements   |"""944 Walnut St | Boston         | MA 02215"""|1         |700.0   |231.0   |
|2023-01-28 14:15:00|afternoon  |141235  |Lightning Charging Cable  |5563319511488|Alimentation|"""185 Maple St  | Portland       | OR 97035"""|1         |14.95   |7.475   |
|2023-01-17 13:33:00|afternoon  |141236  |Wired Headphones          |2113973395220|Vêtements   |"""538 Adams St  | San Francisco  

In [ ]:
from pyspark.sql.functions import hour, col

# Extract the hour from order_date and filter out rows where the hour is between 0 and 5 inclusive
orders_data = orders_data.filter((hour(col("order_date")) < 0) | (hour(col("order_date")) > 5))

orders_data.show(truncate=False)

+-------------------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|order_date         |time_of_day|order_id|product                   |product_id   |category    |purchase_address |quantity_ordered|price_each  |cost_price|turnover|margin  |
+-------------------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|2023-01-22 21:25:00|evening    |141234  |iPhone                    |5638008983335|Vêtements   |"""944 Walnut St | Boston         | MA 02215"""|1         |700.0   |231.0   |
|2023-01-28 14:15:00|afternoon  |141235  |Lightning Charging Cable  |5563319511488|Alimentation|"""185 Maple St  | Portland       | OR 97035"""|1         |14.95   |7.475   |
|2023-01-17 13:33:00|afternoon  |141236  |Wired Headphones          |2113973395220|Vêtements   |"""538 Adams St  | San Francisco  

In [ ]:
from pyspark.sql.functions import to_date

# Convert order_date column from timestamp to date
orders_data = orders_data.withColumn("order_date", to_date(col("order_date")))

orders_data.show(truncate=False)

+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|order_date|time_of_day|order_id|product                   |product_id   |category    |purchase_address |quantity_ordered|price_each  |cost_price|turnover|margin  |
+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|2023-01-22|evening    |141234  |iPhone                    |5638008983335|Vêtements   |"""944 Walnut St | Boston         | MA 02215"""|1         |700.0   |231.0   |
|2023-01-28|afternoon  |141235  |Lightning Charging Cable  |5563319511488|Alimentation|"""185 Maple St  | Portland       | OR 97035"""|1         |14.95   |7.475   |
|2023-01-17|afternoon  |141236  |Wired Headphones          |2113973395220|Vêtements   |"""538 Adams St  | San Francisco  | CA 94016"""|2         |11.99   |5.995   |
|2023-01-0

In [ ]:
# Remove rows where the product column has the value "TV"
orders_data = orders_data.filter(col("product") != "TV")

In [ ]:
# Show the final DataFrame
orders_data.show(truncate=False)

+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|order_date|time_of_day|order_id|product                   |product_id   |category    |purchase_address |quantity_ordered|price_each  |cost_price|turnover|margin  |
+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|2023-01-22|evening    |141234  |iPhone                    |5638008983335|Vêtements   |"""944 Walnut St | Boston         | MA 02215"""|1         |700.0   |231.0   |
|2023-01-28|afternoon  |141235  |Lightning Charging Cable  |5563319511488|Alimentation|"""185 Maple St  | Portland       | OR 97035"""|1         |14.95   |7.475   |
|2023-01-17|afternoon  |141236  |Wired Headphones          |2113973395220|Vêtements   |"""538 Adams St  | San Francisco  | CA 94016"""|2         |11.99   |5.995   |
|2023-01-0

In [ ]:
from pyspark.sql.functions import lower

# Ensure all values in the purchase_state column are lowercase
orders_data = orders_data.withColumn("category", lower(col("category")))

In [ ]:
# Show the final DataFrame
orders_data.show(truncate=False)

+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|order_date|time_of_day|order_id|product                   |product_id   |category    |purchase_address |quantity_ordered|price_each  |cost_price|turnover|margin  |
+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+
|2023-01-22|evening    |141234  |iPhone                    |5638008983335|vêtements   |"""944 Walnut St | Boston         | MA 02215"""|1         |700.0   |231.0   |
|2023-01-28|afternoon  |141235  |Lightning Charging Cable  |5563319511488|alimentation|"""185 Maple St  | Portland       | OR 97035"""|1         |14.95   |7.475   |
|2023-01-17|afternoon  |141236  |Wired Headphones          |2113973395220|vêtements   |"""538 Adams St  | San Francisco  | CA 94016"""|2         |11.99   |5.995   |
|2023-01-0

In [ ]:
from pyspark.sql.functions import substring

# State is the 7th and 8th last characters
orders_data = orders_data.withColumn(
    "state",
    substring("purchase_address", -8, 2)
)

In [ ]:
# Show the final DataFrame
orders_data.show(truncate=False)

+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+-----+
|order_date|time_of_day|order_id|product                   |product_id   |category    |purchase_address |quantity_ordered|price_each  |cost_price|turnover|margin  |state|
+----------+-----------+--------+--------------------------+-------------+------------+-----------------+----------------+------------+----------+--------+--------+-----+
|2023-01-22|evening    |141234  |iPhone                    |5638008983335|vêtements   |"""944 Walnut St | Boston         | MA 02215"""|1         |700.0   |231.0   |al   |
|2023-01-28|afternoon  |141235  |Lightning Charging Cable  |5563319511488|alimentation|"""185 Maple St  | Portland       | OR 97035"""|1         |14.95   |7.475   |Ma   |
|2023-01-17|afternoon  |141236  |Wired Headphones          |2113973395220|vêtements   |"""538 Adams St  | San Francisco  | CA 94016"""|2         

In [ ]:
# Path to save the Parquet file
parquet_path = '/orders_data.parquet'

# Save DataFrame as Parquet
orders_data.write.mode('overwrite').parquet(parquet_path)
